# Few-shot

In [1]:
%pip install langchain-openai langchain-core langchain-community 

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") or getpass.getpass("Enter OpenAI API Key: ")

In [3]:
from langchain_openai import ChatOpenAI

openai_model = "gpt-4.1"
# For normal accurate responses
llm = ChatOpenAI(temperature=0.0, model=openai_model)

In [4]:
user_prompt = """
    Desired Workflow: {desired_workflow}
    List of Devices as WoT Thing Descriptions: \n {thing_directory}
    """

In [5]:
import utils

from langchain_core.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate, 
    HumanMessagePromptTemplate
)

prompt_template = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(utils.system_prompt),
    HumanMessagePromptTemplate.from_template(user_prompt)
])

In [6]:
prompt_template.input_variables

['desired_workflow', 'thing_directory']

In [7]:
example_prompt = ChatPromptTemplate.from_messages([
    ("human", user_prompt),
    ("ai", "{generated_workflow}"),
])

In [8]:
examples = [
    {
        "desired_workflow": utils.smart_home_system_description,
        "thing_directory": utils.smart_home_td_simple,
        "generated_workflow": utils.smart_home_expected_output
    },
    {
        "desired_workflow": utils.manufacturing_system_description,
        "thing_directory": utils.manufacturing_td_simple,
        "generated_workflow": utils.manufacturing_expected_output
    }
]


In [9]:
utils.manufacturing_system_description

'\n  When a quality, temperature, or vibration alert is made, send this data for analysis. Once data analysis is complete log the result as well as analysing it for indications that maintenance is needed. If maintenance is needed, alert the manager of this. Additionally, send the result of data analysis to be analysed for efficiency and alert the manager of this too.\n\n  Send quality alerts directly to the manager.\n\n  When resources are used, alert the manager of the usage, and log it in the database.\n'

In [10]:
from langchain_core.prompts import FewShotChatMessagePromptTemplate

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples
)

print(few_shot_prompt.format())

Human: 
    Desired Workflow: 
  Blink LEDs when washing machine cycle has finished. 

  Turn on the main room light when motion is detected in that room. 
  
  When the door bell is pressed, reduce the speaker volume, make the smart assistant alert the homeowner of the doorbell, return the speaker’s volume. 
  
  When morning alarm triggers, turn heating on to 30 degrees C for 20mins.

    List of Devices as WoT Thing Descriptions: 
 
  [{"@context":["http://www.w3.org/ns/td"],"@type":["Thing"],"title":"Alarm","securityDefinitions":{"no_sec":{"scheme":"nosec"}},"security":"no_sec","actions":{},"properties":{},"events":{"alarmRinging":{"title":"Alarm Ringing","description":"This alarm has started ringing","subscription":{},"data":{"type":"null"},"cancellation":{},"forms":[]}}},{"@context":["http://www.w3.org/ns/td"],"@type":["Thing"],"title":"DoorBell","securityDefinitions":{"no_sec":{"scheme":"nosec"}},"security":"no_sec","actions":{},"properties":{},"events":{"bellRung":{"title":"Bel

In [11]:
few_shot_prompt

FewShotChatMessagePromptTemplate(examples=[{'desired_workflow': '\n  Blink LEDs when washing machine cycle has finished. \n\n  Turn on the main room light when motion is detected in that room. \n  \n  When the door bell is pressed, reduce the speaker volume, make the smart assistant alert the homeowner of the doorbell, return the speaker’s volume. \n  \n  When morning alarm triggers, turn heating on to 30 degrees C for 20mins.\n', 'thing_directory': '\n  [{"@context":["http://www.w3.org/ns/td"],"@type":["Thing"],"title":"Alarm","securityDefinitions":{"no_sec":{"scheme":"nosec"}},"security":"no_sec","actions":{},"properties":{},"events":{"alarmRinging":{"title":"Alarm Ringing","description":"This alarm has started ringing","subscription":{},"data":{"type":"null"},"cancellation":{},"forms":[]}}},{"@context":["http://www.w3.org/ns/td"],"@type":["Thing"],"title":"DoorBell","securityDefinitions":{"no_sec":{"scheme":"nosec"}},"security":"no_sec","actions":{},"properties":{},"events":{"bellRu

In [12]:
prompt_template = ChatPromptTemplate.from_messages([
    ("system", utils.system_prompt),
    few_shot_prompt,
    ("user", user_prompt),
])

In [13]:
prompt_template.input_variables

['desired_workflow', 'thing_directory']

In [14]:
pipeline = (
    {
        "thing_directory": lambda x: x["thing_directory"],
        "desired_workflow": lambda x: x["desired_workflow"]
    }
    | prompt_template
    | llm
    | {"generated_workflow": lambda x: x.content}
)

In [15]:
response = pipeline.invoke({
    "thing_directory": utils.smart_aquarium_td_simple,
    "desired_workflow": utils.smart_aquarium_system_description
})

In [17]:
# formatting the response

import json

# Parse twice: first to extract string, second to get proper JSON
workflow_str = response['generated_workflow']
workflow = json.loads(workflow_str)

# Now `workflow` is a list of dicts — ready for Node-RED
print(json.dumps(workflow, indent=2))

[
  {
    "id": "a1e1b1b1b1b1b1b1",
    "type": "tab",
    "label": "Aquarium Automation",
    "disabled": false,
    "info": "",
    "env": []
  },
  {
    "id": "ph_event",
    "type": "system-event-node",
    "z": "a1e1b1b1b1b1b1b1",
    "name": "pH Level Event",
    "thingDirectoryURI": "http://localhost:8080/thingdirectory",
    "thingEvent": "pHLevel",
    "thingEventValue": "{\"uri\":\"http://localhost:8080/tanksensors\",\"output\":{\"type\":\"number\"},\"description\":\"A periodic report of the pH level of the tank's water\",\"title\":\"pH Level\",\"event\":\"pHLevel\"}",
    "outputToMsg": true,
    "redeploy": "true",
    "x": 120,
    "y": 60,
    "wires": [
      [
        "ph_range_check"
      ]
    ]
  },
  {
    "id": "ph_range_check",
    "type": "function",
    "z": "a1e1b1b1b1b1b1b1",
    "name": "Check pH Range",
    "func": "const minPH = 6.5;\nconst maxPH = 7.5;\nif (msg.payload < minPH) {\n    msg.chemical = \"crushed coral\";\n    msg.amount = 1; // adjust as ne